In [1]:
import os

In [2]:
%pwd

'/Users/muratisiklan/Desktop/Random-Project/research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL : str
    local_data_file:Path
    unzip_dir:Path


In [5]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_dirs

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFING_FILE_PATH
                 ,params_filepath = PARAMS_FILE_PATH) -> None:
        """takes config and params file path 
        creates dirs accordingly...

        Args:
            config_filepath (_type_, optional): _description_. Defaults to CONFING_FILE_PATH.
            params_filepath (_type_, optional): _description_. Defaults to PARAMS_FILE_PATH.
        """
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dirs([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_dirs([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

    

In [7]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier.logging import logger
from src.cnnClassifier.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig) -> None:
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with info --> \n{headers}")
        else:
            logger.info(f"File Already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self)->None:
        """zip_file_path: str
        Extracts zip file into data directory
        returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with(zipfile.ZipFile(self.config.local_data_file,"r")) as zip_ref:
            zip_ref.extractall(unzip_path)


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e